<h1>Abgabe</h1>

In [1]:
!pip install yfinance
!pip install pandas


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Aktienliste für das Training
stocks = ["AAPL", "MSFT", "GOOGL"]

# Daten für jede Aktie herunterladen und in einer Liste speichern
data_list = []
for stock in stocks:
    data = yf.download(stock, start="2010-01-01", end="2022-12-31")
    data['Stock'] = stock
    data_list.append(data)

# Kombinieren aller Aktien in einem DataFrame
data_all = pd.concat(data_list)

# Erste Ansicht der kombinierten Daten
print(data_all.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                Open      High       Low     Close  Adj Close     Volume Stock
Date                                                                          
2010-01-04  7.622500  7.660714  7.585000  7.643214   6.454507  493729600  AAPL
2010-01-05  7.664286  7.699643  7.616071  7.656429   6.465666  601904800  AAPL
2010-01-06  7.656429  7.686786  7.526786  7.534643   6.362820  552160000  AAPL
2010-01-07  7.562500  7.571429  7.466071  7.520714   6.351056  477131200  AAPL
2010-01-08  7.510714  7.571429  7.466429  7.570714   6.393280  447610800  AAPL


In [3]:
# Funktion zur Berechnung der technischen Indikatoren
def calculate_indicators(data):
    data['SMA_20'] = data['Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()
    data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['HMA_20'] = HMA(data['Close'], 20)
    data['HMA_50'] = HMA(data['Close'], 50)
    return data

# Anwendung der Indikatoren auf alle Aktien
data_all = data_all.groupby('Stock').apply(calculate_indicators)

# Daten nach der Berechnung der Indikatoren ansehen
print(data_all.head())


NameError: name 'HMA' is not defined

In [ ]:
# Feature Engineering: Weitere Indikatoren und Zielvariable hinzufügen
def add_features(data):
    data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)
    return data

data_all = data_all.groupby('Stock').apply(add_features)

# Dropping NaN Werte nach Indikatorberechnung
data_all = data_all.dropna()

# Features und Zielvariable definieren
features = ['SMA_20', 'SMA_50', 'EMA_20', 'EMA_50', 'HMA_20', 'HMA_50']
X = data_all[features]
y = data_all['Target']

# Training und Test-Datensätze aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Random Forest Modell trainieren
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Modell auf Testdaten evaluieren
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
# Daten für eine einzelne Aktie zum Testen herunterladen
test_data = yf.download("AAPL", start="2023-01-01", end="2024-01-01")
test_data = calculate_indicators(test_data)
test_data = add_features(test_data)
test_data = test_data.dropna()

# Vorhersagen treffen
X_single = test_data[features]
predictions = model.predict(X_single)

# Ergebnisse visualisieren
plt.figure(figsize=(14, 7))
plt.plot(test_data.index, test_data['Close'], label='Close Price', color='black')
plt.scatter(test_data.index[predictions == 1], test_data['Close'][predictions == 1], label='Buy Signal', marker='^', color='blue')
plt.scatter(test_data.index[predictions == 0], test_data['Close'][predictions == 0], label='Sell Signal', marker='v', color='orange')
plt.title('Buy and Sell Signals for AAPL')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
